In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions.{datediff, hour, to_date, udf,lower,concat,lit, from_unixtime,format_number,when}

Intitializing Scala interpreter ...

Spark Web UI available at http://10.78.180.192:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1572344605021)
SparkSession available as 'spark'


import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions.{datediff, hour, to_date, udf, lower, concat, lit, from_unixtime, format_number, when}


In [2]:
val df : DataFrame = spark.read
      .option("header",true)
      .option("inferSchema","true")
      .csv("data/train_clean.csv")

df: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 12 more fields]


+--------------+--------------------+--------------------+-------+--------------------+---------------------+-------+--------+----------+----------------+----------+-----------+-------------+------------+
|    project_id|                name|                desc|   goal|            keywords|disable_communication|country|currency|  deadline|state_changed_at|created_at|launched_at|backers_count|final_status|
+--------------+--------------------+--------------------+-------+--------------------+---------------------+-------+--------+----------+----------------+----------+-----------+-------------+------------+
|kkst1451568084| drawing for dollars|I like drawing pi...|   20.0| drawing-for-dollars|                False|     US|     USD|1241333999|      1241334017|1240600507| 1240602723|            3|           1|
|kkst1474482071|Sponsor Dereck Bl...|I  Dereck Blackbu...|  300.0|sponsor-dereck-bl...|                False|     US|     USD|1242429000|      1242432018|1240960224| 1240975592|   

In [4]:

    print(s"Nombre de ligne :  ${df.count}")
    println(s"Nombre de colonnes : ${df.columns.length}")
    //df.show()
    //df.printSchema()
    val dfCasted : DataFrame = df
      .withColumn("goal", df("goal").cast("Int"))
      .withColumn("deadline" , df("deadline").cast("Int"))
      .withColumn("state_changed_at", df("state_changed_at").cast("Int"))
      .withColumn("created_at", df("created_at").cast("Int"))
      .withColumn("launched_at", df("launched_at").cast("Int"))
      .withColumn("backers_count", df("backers_count").cast("Int"))
      .withColumn("final_status", df("final_status").cast("Int"))

    


Nombre de ligne :  108129Nombre de colonnes : 14


dfCasted: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 12 more fields]


In [5]:

    dfCasted
      .select("goal", "backers_count", "final_status")
      .describe()
      .show
    /*
    dfCasted.groupBy("disable_communication").count.orderBy($"count".desc).show(20)
    dfCasted.groupBy("country").count.orderBy($"count".desc).show(20)
    dfCasted.groupBy("currency").count.orderBy($"count".desc).show(20)
    dfCasted.select("deadline").dropDuplicates.show()
    dfCasted.groupBy("state_changed_at").count.orderBy($"count".desc).show(20)
    dfCasted.groupBy("backers_count").count.orderBy($"count".desc).show(20)
    dfCasted.select("goal", "final_status").show(30)
    dfCasted.groupBy("country", "currency").count.orderBy($"count".desc).show(50)
    */
    val df2: DataFrame = dfCasted.drop("disable_communication")

    val dfNoFutur : DataFrame = df2.drop("backers_count","state_changed_at")


+-------+-----------------+-------------------+-------------------+
|summary|             goal|      backers_count|       final_status|
+-------+-----------------+-------------------+-------------------+
|  count|           107615|             108128|             108128|
|   mean|36839.03430748502|  6434187.413250962| 1052360.7834973366|
| stddev|974215.3015529711|9.324061726649424E7|3.776049940184161E7|
|    min|                0|                  0|                  0|
|    max|        100000000|         1430423170|         1428977971|
+-------+-----------------+-------------------+-------------------+



df2: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 11 more fields]
dfNoFutur: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 9 more fields]


In [7]:

    df.filter($"country" === "False")
      .groupBy("currency")
      .count
      .orderBy($"count".desc)
      .show(50)

  def cleanCountry(country: String, currency: String): String = {
    if (country == "False" )
      currency
    else
      country
  }

  def cleanCurrency(currency: String): String = {
    if (currency != null && currency.length != 3)
      null
    else
      currency
  }

    val cleanCountryUdf = udf(cleanCountry _)
    val cleanCurrencyUdf = udf(cleanCurrency _)

    val dfCountry: DataFrame = dfNoFutur
      .withColumn("country2", cleanCountryUdf($"country", $"currency"))
      .withColumn("currency2", cleanCurrencyUdf($"currency"))
      .drop("country", "currency")


+--------+-----+
|currency|count|
+--------+-----+
|      US|  405|
|      GB|   13|
|      AU|    3|
|      CA|    3|
|      NL|    2|
|      NZ|    1|
|      NO|    1|
+--------+-----+



cleanCountry: (country: String, currency: String)String
cleanCurrency: (currency: String)String
cleanCountryUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function2>,StringType,Some(List(StringType, StringType)))
cleanCurrencyUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))
dfCountry: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 9 more fields]


In [10]:

    //dfCountry.groupBy("final_status").count.orderBy($"count".desc).show()
    // Ici nous allons séléctionner que les campagens ayant un final-status à 0 ou 1.
    // On pourrait toutefois tester en mettant toutes les autres valeurs à 0
    // en considérant que les campagnes qui ne sont pas un Success sont un Fail.
    val dfFinalStatus : DataFrame = dfCountry
      .withColumn("final_status", when($"final_status"===0 || $"final_status"===1,$"final_status").otherwise(null))
        .filter($"final_status".isNotNull)
    //dfFinalStatus.groupBy("final_status").count.orderBy($"count".desc).show()
    //dfFinalStatus.printSchema()
   // dfFinalStatus.show()

    val dfNbDays : DataFrame = dfFinalStatus
      .withColumn("deadline2",from_unixtime($"deadline"))
      .withColumn("launched_at2",from_unixtime($"launched_at"))
      .withColumn("created_at2",from_unixtime($"created_at"))
      .withColumn("days_campaign", datediff($"deadline2",$"launched_at2"))
      .withColumn("hours_prepa", format_number(($"launched_at" - $"created_at")/3600,3))
      .drop("launched_at","created_at","deadline","launched_at2","created_at2","deadline2")

    //dfNbDays.show()
    val dfText : DataFrame = dfNbDays
      .withColumn("desc", lower($"desc"))
      .withColumn("name", lower($"name"))
      .withColumn("keywords", lower($"keywords"))
        .withColumn("text",concat($"name",lit(" "),$"desc",lit(" "),$"keywords"))
        .drop("name","desc","keywords")
    //dfText.show()
    //val cleanNullIntUdf = udf(cleanNullInt _)
    //val cleanNullStringUdf = udf(cleanNullString _)


dfFinalStatus: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 9 more fields]
dfNbDays: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 8 more fields]
dfText: org.apache.spark.sql.DataFrame = [project_id: string, goal: int ... 6 more fields]


In [80]:

    val dfCleanNull : DataFrame = dfText
      .withColumn("days_campaign",when($"days_campaign".isNull,-1).otherwise($"days_campaign"))
      .withColumn("goal",when($"goal"isNull, -1).otherwise($"goal"))
      .withColumn("hours_prepa",when($"hours_prepa"isNull,-1).otherwise($"hours_prepa"))
      .withColumn("country2",when($"country2"==="True","unknown").otherwise($"country2"))
      .withColumn("currency2",when($"currency2".isNull,"unknown").otherwise($"currency2"))


dfCleanNull: org.apache.spark.sql.DataFrame = [project_id: string, goal: int ... 6 more fields]


+--------+-----+
|country2|count|
+--------+-----+
| unknown|    1|
|      DE|    1|
|      IE|  111|
|      NO|  113|
|      DK|  196|
|      SE|  240|
|      NZ|  354|
|      NL|  704|
|      AU| 1877|
|      CA| 3735|
|      GB| 8746|
|      US|91607|
+--------+-----+

+---------+-----+
|currency2|count|
+---------+-----+
|  unknown|   70|
|      NOK|  113|
|      DKK|  196|
|      SEK|  240|
|      NZD|  354|
|      EUR|  814|
|      AUD| 1877|
|      CAD| 3733|
|      GBP| 8743|
|      USD|91545|
+---------+-----+



In [81]:

//Environ 22000 ligne avec des hours_prepa negatifs et que 37 en dessous de -10 370 en dessous de -5 => solution de les mettre tous à 0
    print("preclean",dfCleanNull.where(($"hours_prepa" <0) ).count())
    val dfCleaned : DataFrame = dfCleanNull
        .withColumn("hours_prepa", when($"hours_prepa" < 0, -1).otherwise($"hours_prepa"))
    print("ceci est un test post clean ",dfCleaned.filter(($"hours_prepa" <0) ).count())

(preclean,37)(ceci est un test post clean ,37)

dfCleaned: org.apache.spark.sql.DataFrame = [project_id: string, goal: int ... 6 more fields]


In [82]:
dfCleaned.where($"goal" < 0).count()

res70: Long = 70


In [83]:
dfCleaned.where($"goal".isNull).count()

res71: Long = 0


In [85]:
dfCleaned.where($"days_campaign" < 0).count()

res73: Long = 70


In [79]:
dfCleaned.where($"hours_prepa".isNull).count()

res68: Long = 0


# TP 3 

## Utilisation des données textuelles

In [2]:
import org.apache.spark.ml.feature.{IDF, Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, StringIndexer, OneHotEncoder, VectorAssembler}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression

import org.apache.spark.ml.feature.{IDF, Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, StringIndexer, OneHotEncoder, VectorAssembler}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression


In [3]:
 val df : DataFrame = spark.read
      .parquet("data/prepared_trainingset")

df: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 12 more fields]


In [24]:
df.show()

+--------------+--------------------+--------------------+------+--------------------+------------+--------+---------+-------------------+-------------------+-------------------+-------------+-----------+--------------------+
|    project_id|                name|                desc|  goal|            keywords|final_status|country2|currency2|          deadline2|        created_at2|       launched_at2|days_campaign|hours_prepa|                text|
+--------------+--------------------+--------------------+------+--------------------+------------+--------+---------+-------------------+-------------------+-------------------+-------------+-----------+--------------------+
| kkst471421639|american options ...|looking to create...|100000|american-options-...|           0|      US|      USD|2014-11-15 17:31:27|2014-10-10 21:23:58|2014-10-16 17:31:27|           30|    140.125|american options ...|
|kkst1098019088|iheadbones bone c...|wireless bluetoot...| 20000|iheadbones-bone-c...|          

In [44]:
df.select($"text".isNull).groupBy($"(text IS NULL)").count().show()

+--------------+------+
|(text IS NULL)| count|
+--------------+------+
|         false|107614|
+--------------+------+



In [4]:
val tokenizer = new RegexTokenizer()
  .setPattern("\\W+")
  .setGaps(true)
  .setInputCol("text")
  .setOutputCol("tokens")

tokenizer: org.apache.spark.ml.feature.RegexTokenizer = regexTok_7fb4a5b35c27


In [5]:
val stopWordsRemover = new StopWordsRemover()
    .setInputCol("tokens")
    .setOutputCol("tokensWOstopwords")

stopWordsRemover: org.apache.spark.ml.feature.StopWordsRemover = stopWords_0bd2e323e552


In [8]:
val cvModel = new CountVectorizer()
  .setInputCol("tokensWOstopwords")
  .setOutputCol("countedWord")
  .setMinDF(2) //a word has to appear 2 times to be in the vocabulary 
  .fit(stopWordsRemover.transform(tokenizer.transform(df)))


cvModel: org.apache.spark.ml.feature.CountVectorizerModel = cntVec_76cd679fe637


In [9]:
val idf = new IDF()
    .setInputCol("countedWord")
    .setOutputCol("tfidf")

idf: org.apache.spark.ml.feature.IDF = idf_f93067bbd302


## Conversion des variables catégorielles en variables numériques

In [10]:
val indexerCountry = new StringIndexer()
  .setInputCol("country2")
  .setOutputCol("country_indexed")

indexerCountry: org.apache.spark.ml.feature.StringIndexer = strIdx_40ee9c48297f


In [11]:
val indexerCurrency = new StringIndexer()
  .setInputCol("currency2")
  .setOutputCol("currency_indexed")

indexerCurrency: org.apache.spark.ml.feature.StringIndexer = strIdx_ea3bcdb400d2


In [12]:
val onehotencoderCountry = new OneHotEncoder()
    .setInputCol("country_indexed")
    .setOutputCol("country_onehot")

onehotencoderCountry: org.apache.spark.ml.feature.OneHotEncoder = oneHot_52327de50c55


In [13]:
val onehotencoderCurrency = new OneHotEncoder()
    .setInputCol("currency_indexed")
    .setOutputCol("currency_onehot")

onehotencoderCurrency: org.apache.spark.ml.feature.OneHotEncoder = oneHot_504154201d02


In [14]:
val assembler = new VectorAssembler()
  .setInputCols(Array("tfidf", "days_campaign", "hours_prepa", "goal", "country_onehot", "currency_onehot"))
  .setOutputCol("features")

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_f13d911a3677


In [15]:
val lr = new LogisticRegression()
  .setElasticNetParam(0.0)
  .setFitIntercept(true)
  .setFeaturesCol("features")
  .setLabelCol("final_status")
  .setStandardization(true)
  .setPredictionCol("predictions")
  .setRawPredictionCol("raw_predictions")
  .setThresholds(Array(0.7, 0.3))
  .setTol(1.0e-6)
  .setMaxIter(20)

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_b3ddd14557b3


In [16]:
val splits = df.randomSplit(Array(0.9, 0.1))
val training = splits(0).cache()
val test = splits(1)

splits: Array[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = Array([project_id: string, name: string ... 12 more fields], [project_id: string, name: string ... 12 more fields])
training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [project_id: string, name: string ... 12 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [project_id: string, name: string ... 12 more fields]


In [17]:
val pipeline = new Pipeline()
  .setStages(Array(tokenizer, stopWordsRemover,cvModel,idf, indexerCountry,indexerCurrency,
                   onehotencoderCountry, onehotencoderCurrency, assembler, lr))

pipeline: org.apache.spark.ml.Pipeline = pipeline_2fbd37ef13cb


In [31]:
val model = pipeline.fit(training)

model: org.apache.spark.ml.PipelineModel = pipeline_2fbd37ef13cb


In [168]:
df.show()

+--------------+--------------------+--------------------+------+--------------------+------------+--------+---------+-------------------+-------------------+-------------------+-------------+-----------+--------------------+
|    project_id|                name|                desc|  goal|            keywords|final_status|country2|currency2|          deadline2|        created_at2|       launched_at2|days_campaign|hours_prepa|                text|
+--------------+--------------------+--------------------+------+--------------------+------------+--------+---------+-------------------+-------------------+-------------------+-------------+-----------+--------------------+
| kkst471421639|american options ...|looking to create...|100000|american-options-...|           0|      US|      USD|2014-11-15 17:31:27|2014-10-10 21:23:58|2014-10-16 17:31:27|           30|    140.125|american options ...|
|kkst1098019088|iheadbones bone c...|wireless bluetoot...| 20000|iheadbones-bone-c...|          

In [189]:
val predic = model.transform(test)

predic: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 24 more fields]


In [18]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator


In [19]:
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("final_status")
  .setPredictionCol("predictions")
  .setMetricName("f1")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_1318d9fc65c6


In [196]:
val result = evaluator.evaluate(predic)

result: Double = 0.6249634785483382


In [197]:
val sameModel = PipelineModel.load("spark-logistic-regression-model")

sameModel: org.apache.spark.ml.PipelineModel = pipeline_311b3948c71b


In [205]:
val paramGrid = new ParamGridBuilder()
  .addGrid(cvModel.minDF, Array(55.0,75.0,95.0))
  .addGrid(lr.regParam, Array(10e-8, 10e-6, 10e-4, 10e-2))
  .build()

val lrtv = TrainValidationSplit()
    .setEstimator(pipeline)
    .setEstimatorParamMaps(paramGrid)
    .setEvaluator(evaluator)

val modelGrid = lrtv.fit(training)

<console>: 70: error: org.apache.spark.ml.tuning.TrainValidationSplit.type does not take parameters

In [21]:
import org.apache.spark.ml.feature.{IDF, Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, StringIndexer, OneHotEncoder, VectorAssembler,CountVectorizerModel}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit,TrainValidationSplitModel, CrossValidator, CrossValidatorModel}

import org.apache.spark.ml.feature.{IDF, Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, StringIndexer, OneHotEncoder, VectorAssembler, CountVectorizerModel}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, TrainValidationSplitModel, CrossValidator, CrossValidatorModel}


In [23]:

val paramGrid = new ParamGridBuilder()
      .addGrid(cvModel.minDF, Array(55.0,75.0,95.0))
      .addGrid(lr.regParam, Array(10e-8, 10e-6, 10e-4, 10e-2))
      .build()
val lrcv = new CrossValidator()
      .setEstimator(pipeline)
      .setEstimatorParamMaps(paramGrid)
      .setEvaluator(evaluator)
      .setNumFolds(5)

    val modelGridCV = lrcv.fit(training)

org.apache.spark.SparkException:  Exception thrown in awaitResult: